# Auto-provision a new user

- Create a home directory
- Create NFS export
- Create SMB share
- Set up daily snapshots
- Set up replication and replicate data

In [72]:
# Set up rest client
import os
import sys
import traceback

from qumulo.rest_client import RestClient
from qumulo.rest.nfs import NFSRestriction

In [73]:
cluster = 'product'
base_dir = 'users-replicated'
user_name = 'tommy-at-qumulo'
dest_ip = '10.120.200.31'
dest_dir = 'users-backup'
full_path = '/'+ base_dir + '/' + user_name

In [74]:
rc = RestClient(cluster, 8000)
rc.login("admin", "admin")

rc_dest = RestClient(dest_ip, 8000)
rc_dest.login("admin", "admin");

In [75]:
def create_dir(rc, name, dir_path='/'):
    try:
        rc.fs.create_directory(name = name, dir_path = dir_path)
    except:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        print("Exception: %s" % exc_value)

In [76]:
# Create base user directory, if it doesn't already exist
create_dir(rc, name=base_dir, dir_path='/')

Exception: Error 409: fs_entry_exists_error: { ino=252355565 }


### Create directory

In [77]:
dir_res = rc.fs.create_directory(name=user_name, dir_path='/'+ base_dir)
print("Directory '%s' created with id: %s" % (full_path, dir_res['file_number']))
dir_id = dir_res['file_number']

Directory '/users-replicated/tommy-at-qumulo' created with id: 306365198


### Create 20GB Quota

In [78]:
quota_res = rc.quota.create_quota(id_ = dir_id, limit_in_bytes = 20000000000)

### Create NFS export

In [79]:
nfs_res = rc.nfs.nfs_add_share(export_path = '/' + user_name,
                        fs_path = full_path,
                        description = "%s home directory" % user_name,
                        restrictions = [NFSRestriction({
                                            'read_only': False, 
                                            'host_restrictions': [],
                                            'user_mapping': 'NFS_MAP_NONE', 
                                            'map_to_user_id': '0'})]
                    )
print("NFS export created: %s with id %s" % (full_path, nfs_res['id']))

NFS export created: /users-replicated/tommy-at-qumulo with id 8909


### Create SMB share

In [80]:
smb_res = rc.smb.smb_add_share(share_name = user_name, 
                           fs_path = full_path, 
                           description = "%s home directory" % user_name
                          )
print("SMB share created: %s with id %s" % (full_path, smb_res['id']))

SMB share created: /users-replicated/tommy-at-qumulo with id 8910


### Set up snapshot policy

In [81]:
snap_res = rc.snapshot.create_policy(name = "User %s" % user_name, 
                                schedule_info = {"creation_schedule":
                                                 {"frequency":"SCHEDULE_DAILY_OR_WEEKLY",
                                                  "hour":2,"minute":15,
                                                  "on_days":["MON","TUE","WED","THU","FRI","SAT","SUN"],
                                                  "timezone":"America/Los_Angeles"},
                                                 "expiration_time_to_live":"7days"
                                                },
                                directory_id = str(dir_id),
                                enabled = True)
print("Snapshot policy created with id %s" % snap_res['id'])

Snapshot policy created with id 9


### Set up replication

In [82]:
replic_res = rc.replication.create_relationship(source_path = full_path,
                                    target_path = '/%s/%s' % (dest_dir,  user_name),
                                    address = dest_ip,
                                  )

create_dir(rc_dest, name=dest_dir, dir_path='/')
create_dir(rc_dest, name=user_name, dir_path='/' + dest_dir + '/')

replic_auth_res = rc_dest.replication.authorize(relationship_id = replic_res['id'])

print("Replication relationship status: %s" % replic_auth_res['state'])

Exception: Error 409: fs_entry_exists_error: { ino=3700990136 }
Replication relationship status: ESTABLISHED


In [83]:
# mount the share. Create a file.
# sudo mount product:/tommy-at-qumulo /mnt/tommyatqumulo
# dd if=/dev/zero of=/mnt/tommyatqumulo/file1.txt bs=1000000 count=1000

In [84]:
# run this after writing the file
rc.replication.replicate(relationship = replic_res['id'])

{u'monitor_uri': u'/v1/replication/source-relationships/eea3f5b4-9631-4472-b598-f40d8298ecff/status'}

### Clean up everything

In [85]:
rc.quota.delete_quota(id_ = quota_res['id'])
rc.snapshot.delete_policy(policy_id = snap_res['id'])
rc.replication.delete_relationship(relationship_id = replic_res['id'])
rc_dest.fs.delete_tree(path = '/users-backup/%s' % user_name)
rc.smb.smb_delete_share(id_ = smb_res['id'])
rc.nfs.nfs_delete_share(id_ = nfs_res['id'])
rc.fs.delete_tree(path = full_path)
print("Everything is cleaned up!")

Everything is cleaned up!
